# Warsaw Pilot EEG/HRV Hyperscanning Analysis

**SYNCCIN 2025 Summer School**


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jzygierewicz/SYNCCIN_2025_summer_school/blob/main/Warsaw_Pilot_EEG_Hyperscanning_Analysis.ipynb)

---

## **Learning Objectives**


This notebook demonstrates EEG connectivity analysis using real hyperscanning data from child-caregiver dyads. The analysis includes:

- Data loading and preprocessing
- Directed Transfer Function (DTF) analysis for connectivity in HRV in a hyperscanning settings
- Directed Transfer Function (DTF) analysis for connectivity in EEG - each member of the diad separately
- Selection of effect of interest
- Multimodal connectivity between EEG theta activity and heart rate variability in the hyperscannig settings

**Data Source**: Real hyperscanning experiment with child-caregiver pairs watching movies and during talk sessions.

## Setup and Data Download

First, we need to download the required data files and install dependencies.

In [ ]:
# Install required packages for Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install xmltodict neurokit2 plotly

# Download data files from the university server
import os
import urllib.request

# Create data directory
os.makedirs('W_009', exist_ok=True)

# Base URL for data files
base_url = "https://www.fuw.edu.pl/~jarekz/HeidelbergSchool/W_009/"

# List of files to download
files_to_download = [
    "W_009.obci.raw",
    "W_009.obci.xml", 
    "W_009.obci.tag",
    "2025_07_03_15_50_W_009_stage_timings.txt",
    "W_009_03_07_2025_14_56_A839C92B_ECG.csv",
    "W_009_03_07_2025_14_56_A839C92B_IBI.csv",
    "W_009_03_07_2025_14_56_A83E1E24_ECG.csv",
    "W_009_03_07_2025_14_56_A83E1E24_IBI.csv"
]

print("Downloading W_009 dataset...")
for filename in files_to_download:
    file_path = f"W_009/{filename}"
    if not os.path.exists(file_path):
        print(f"Downloading {filename}...")
        try:
            urllib.request.urlretrieve(f"{base_url}{filename}", file_path)
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed to download {filename}: {e}")
    else:
        print(f"{filename} already exists")

print("Data download complete!")

In [ ]:
# Download required Python modules
modules_url = "https://raw.githubusercontent.com/jzygierewicz/SYNCCIN_2025_summer_school/main/"

modules_to_download = ["mtmvar.py", "utils.py"]

for module in modules_to_download:
    if not os.path.exists(module):
        print(f"Downloading {module}...")
        try:
            urllib.request.urlretrieve(f"{modules_url}{module}", module)
            print(f"Downloaded {module}")
        except Exception as e:
            print(f"Failed to download {module}: {e}")
    else:
        print(f"{module} already exists")

## Import Libraries

Import all necessary libraries for data processing, visualization, and connectivity analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.signal import butter, iirnotch, filtfilt, sosfiltfilt, decimate, hilbert, welch
from scipy.stats import zscore
from mtmvar import (mvar_criterion, AR_coeff, mvar_H, mvar_plot, mvar_plot_dense, 
                   mvar_spectra, DTF_multivariate, multivariate_spectra, graph_plot)
from utils import (load_warsaw_pilot_data, scan_for_events, filter_warsaw_pilot_data, 
                  get_IBI_signal_from_ECG_for_selected_event, get_data_for_selected_channel_and_event,
                  plot_EEG_channels, plot_EEG_channels_pl, overlay_EEG_channels_hyperscanning_pl, 
                  overlay_EEG_channels_hyperscanning)

print("All libraries imported successfully!")

## Configuration and Data Loading

Set up analysis parameters and load the raw EEG/ECG data.

In [ ]:
# Configuration
folder = './W_009/'
file = 'W_009.obci'
selected_events = ['Movie_1']  # Focus on Movie_1 event for this analysis

print(f"Loading data from {folder}{file}...")

### Read and preprocess the data

In [ ]:
# Read raw data 
data = load_warsaw_pilot_data(folder, file, plot=False)

# Detect events from diode signal
events = scan_for_events(data, plot=False)

# Apply filtering and preprocessing
filtered_data = filter_warsaw_pilot_data(data)

print("Data loading and preprocessing complete!")
print(f"Data shape: {filtered_data['data'].shape}")
print(f"Sampling frequency (EEG): {filtered_data['Fs_EEG']} Hz")
print(f"Events detected: {events}")

## Data Exploration

Examine the structure of the filtered data and visualize ECG/IBI signals.

In [ ]:
print("=== Data Overview ===")
print(f"Filtered data shape: {filtered_data['data'].shape}")
print(f"Child EEG channels: {filtered_data['EEG_channels_ch']}")
print(f"Caregiver EEG channels: {filtered_data['EEG_channels_cg']}")
print(f"Events detected: {events}")

### Interactive EEG channel visualization using Plotly

In [ ]:
plot_EEG_channels_pl(filtered_data, events, filtered_data['EEG_channels_ch'], 
                    title='Filtered Child EEG Channels (Interactive)', renderer='notebook')
plot_EEG_channels_pl(filtered_data, events, filtered_data['EEG_channels_cg'], 
                    title='Filtered Caregiver EEG Channels (Interactive)', renderer='notebook')

## Select the events to analyse

In [ ]:
selected_events = ['Movie_1'] # you may also like to try other events, they can be added to the list: 'Movie_2', 'Movie_3', 'Talk_1', 'Talk_2'

## Heart Rate Variability DTF Analysis

Analyze connectivity between child and caregiver heart rate variability using DTF.

In [ ]:
# Frequency vector for DTF estimation
f = np.arange(0.01, 1, 0.01)

for event in selected_events:
    if event in events:
        print(f"\nAnalyzing event: {event}")
        
        # Extract IBI signals for the selected event
        IBI_ch_interp, IBI_cg_interp, t_IBI = get_IBI_signal_from_ECG_for_selected_event(
            filtered_data, events, event, plot=False, label=f'IBI signals for {event}')
        
        # Z-score normalize the IBI signals
        IBI_ch_interp = zscore(IBI_ch_interp)
        IBI_cg_interp = zscore(IBI_cg_interp)
        
        # Construct data array for DTF analysis
        data = np.zeros((2, 60 * filtered_data['Fs_IBI']))
        data[0, :] = IBI_ch_interp
        data[1, :] = IBI_cg_interp
        
        # Estimate multivariate spectra and DTF
        S = multivariate_spectra(data, f, Fs=filtered_data['Fs_IBI'], max_p=15, p_opt=None, crit_type='AIC')
        DTF = DTF_multivariate(data, f, Fs=filtered_data['Fs_IBI'], max_p=15, p_opt=None, crit_type='AIC')
        
        # Plot DTF results
        mvar_plot(S, DTF, f, 'From', 'To', ['Child', 'Caregiver'], f'DTF {event}', 'sqrt')
        plt.show()

## EEG Connectivity Analysis

Analyze EEG connectivity within child and caregiver separately using DTF.

In [ ]:
# Frequency vector for EEG DTF estimation
f = np.arange(1, 30, 0.5)

# Selected EEG channels for analysis
selected_channels_ch = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'O2']
selected_channels_cg = ['Fp1_cg', 'Fp2_cg', 'F7_cg', 'F3_cg', 'Fz_cg', 'F4_cg', 'F8_cg', 'C3_cg', 'Cz_cg', 'C4_cg', 'P3_cg', 'Pz_cg', 'P4_cg', 'O1_cg', 'O2_cg']

for event in selected_events:
    print(f"\nAnalyzing EEG connectivity for event: {event}")
    
    # Extract EEG data for selected channels and event
    data_ch = get_data_for_selected_channel_and_event(filtered_data, selected_channels_ch, events, event)
    data_cg = get_data_for_selected_channel_and_event(filtered_data, selected_channels_cg, events, event)
    
    # Visualize EEG channels
    overlay_EEG_channels_hyperscanning(data_ch, data_cg, filtered_data['channels'], event, 
                                        selected_channels_ch, selected_channels_cg, 
                                        title='Filtered EEG Channels - Hyperscanning')
    plt.show()
    
    # Interactive visualization
    overlay_EEG_channels_hyperscanning_pl(data_ch, data_cg, filtered_data['channels'], event, 
                                            selected_channels_ch, selected_channels_cg, 
                                            title='Filtered EEG Channels - Hyperscanning (Interactive)', 
                                            renderer='notebook')

 # Continue with DTF analysis for EEG separately for each person

In [ ]:
for event in selected_events:
    print(f"\nDTF analysis for {event}")
    
    data_ch = get_data_for_selected_channel_and_event(filtered_data, selected_channels_ch, events, event)
    data_cg = get_data_for_selected_channel_and_event(filtered_data, selected_channels_cg, events, event)
    
    # Fixed model order for consistency
    p_opt = 9
    
    # Child EEG DTF
    print("Computing DTF for child EEG channels...")
    S = multivariate_spectra(data_ch, f, Fs=filtered_data['Fs_EEG'], max_p=15, p_opt=p_opt, crit_type='AIC')
    DTF = DTF_multivariate(data_ch, f, Fs=filtered_data['Fs_EEG'], max_p=15, p_opt=p_opt, crit_type='AIC')
    mvar_plot_dense(S, DTF, f, 'From', 'To', selected_channels_ch, f'DTF child {event}', 'sqrt')
    plt.show()
    
    # Caregiver EEG DTF
    print("Computing DTF for caregiver EEG channels...")
    S = multivariate_spectra(data_cg, f, Fs=filtered_data['Fs_EEG'], max_p=15, p_opt=p_opt, crit_type='AIC')
    DTF = DTF_multivariate(data_cg, f, Fs=filtered_data['Fs_EEG'], max_p=15, p_opt=p_opt, crit_type='AIC')
    mvar_plot_dense(S, DTF, f, 'From', 'To', selected_channels_cg, f'DTF caregiver {event}', 'sqrt')
    plt.show()

## Multimodal EEG-HRV Connectivity Analysis in Hyperscanning

Analyze connectivity between EEG theta activity (Fz electrode) and heart rate variability for child and caregiver together.

### First let's check that there is some activity in the selected electrode and frequency band.

In [ ]:
# Selected channels for theta analysis
selected_channels_ch = ['Fz']
selected_channels_cg = ['Fz_cg']

# define the chosen band, e.g., theta band (5-7.5 Hz)
lowcut = 5.0
highcut = 7.5

for event in selected_events:
    print(f"\nAnalyzing multimodal connectivity for event: {event}")
    
    # Extract Fz channel data
    data_ch = get_data_for_selected_channel_and_event(filtered_data, selected_channels_ch, events, event)
    data_cg = get_data_for_selected_channel_and_event(filtered_data, selected_channels_cg, events, event)
    
    # Compute power spectral density
    f_ch, Pxx_ch = welch(data_ch[0, :], fs=filtered_data['Fs_EEG'], nperseg=1024)
    f_cg, Pxx_cg = welch(data_cg[0, :], fs=filtered_data['Fs_EEG'], nperseg=1024)
    
    # Plot power spectrum
    plt.figure(figsize=(12, 6))
    plt.plot(f_ch, Pxx_ch, label='Child Fz channel')
    plt.plot(f_cg, Pxx_cg, label='Caregiver Fz_cg channel')
    plt.title(f'Power Spectrum of {event} for Child and Caregiver Fz channels')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (µV²/Hz)')
    plt.axvspan(lowcut, highcut, color='yellow', alpha=0.5, label='Theta Band (5-7.5 Hz)')
    plt.xlim(0, 30)
    plt.legend()
    plt.grid()
    plt.show()

### Preprocessing: 
- selecting the EEG electrode of interest
- filtering the data in the chosen frequency band
- calculating the instataneous amplitude of the selected band (at this point it is good to check if the envelope looks reasonably)
- downsample the envelope to the same sampling frequency as the interpolated IBI signals
- get the IBI signals
- z-score the signals
- create the multimodal data array 
Finally visualise the prepared set 



In [ ]:
# Design bandpass filter for the chosen band (lowcut, highcut)
sos_theta = butter(4, [lowcut, highcut], btype='band', fs=filtered_data['Fs_EEG'], output='sos')

for event in selected_events:
    print(f"Processing theta band activity for {event}")
    
    data_ch = get_data_for_selected_channel_and_event(filtered_data, selected_channels_ch, events, event)
    data_cg = get_data_for_selected_channel_and_event(filtered_data, selected_channels_cg, events, event)
    
    # Filter in theta band
    data_ch_theta = sosfiltfilt(sos_theta, data_ch[0, :])
    data_cg_theta = sosfiltfilt(sos_theta, data_cg[0, :])
    
    # Extract instantaneous amplitude using Hilbert transform
    data_ch_theta_amp = np.abs(hilbert(data_ch_theta))
    data_cg_theta_amp = np.abs(hilbert(data_cg_theta))
    
    # Plot theta amplitude and filtered signal
    fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    ax[0].set_title(f'Child EEG channel Fz theta amplitude for {event}')
    ax[1].set_title(f'Caregiver EEG channel Fz_cg theta amplitude for {event}')
    ax[0].plot(data_ch_theta_amp, 'r', label='Fz theta amplitude')
    ax[1].plot(data_cg_theta_amp, 'r', label='Fz_cg theta amplitude')
    ax[0].plot(data_ch_theta, 'k', alpha=0.5, label='Fz theta filtered signal')
    ax[1].plot(data_cg_theta, 'k', alpha=0.5, label='Fz_cg theta filtered signal')
    ax[0].set_ylabel('Amplitude (µV)')
    ax[1].set_ylabel('Amplitude (µV)')
    ax[0].legend(loc='upper right')
    ax[1].legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    
    # Downsample theta amplitude to IBI frequency
    data_ch_theta_amp = decimate(data_ch_theta_amp, filtered_data['Fs_EEG']//filtered_data['Fs_IBI'], axis=-1)
    data_cg_theta_amp = decimate(data_cg_theta_amp, filtered_data['Fs_EEG']//filtered_data['Fs_IBI'], axis=-1)
    
    # Z-score normalize
    data_ch_theta_amp = zscore(data_ch_theta_amp)
    data_cg_theta_amp = zscore(data_cg_theta_amp)
    
    # Get IBI signals
    IBI_ch_interp, IBI_cg_interp, t_IBI = get_IBI_signal_from_ECG_for_selected_event(
        filtered_data, events, event, plot=False, label=f'IBI signals for {event}')
    
    # Z-score normalize IBI signals
    IBI_ch_interp = zscore(IBI_ch_interp)
    IBI_cg_interp = zscore(IBI_cg_interp)
    
    # Construct multimodal data array: [Child IBI, Caregiver IBI, Child Fz theta, Caregiver Fz theta]
    DTF_data = np.zeros((4, len(data_ch_theta_amp)))
    DTF_data[0, :] = IBI_ch_interp
    DTF_data[1, :] = IBI_cg_interp
    DTF_data[2, :] = data_ch_theta_amp
    DTF_data[3, :] = data_cg_theta_amp
    
    # Plot multimodal signals
    fig, ax = plt.subplots(4, 1, figsize=(12, 8), sharex=True)
    signals = ['Child IBI signal', 'Caregiver IBI signal', 'Child Fz theta amplitude', 'Caregiver Fz_cg theta amplitude']
    ylabels = ['IBI (ms)', 'IBI (ms)', 'Fz theta amplitude (µV)', 'Fz_cg theta amplitude (µV)']
    colors = ['b', 'b', 'r', 'r']
    
    for i in range(4):
        ax[i].set_title(f'{signals[i]} for {event}')
        ax[i].plot(t_IBI, DTF_data[i, :], colors[i], label=signals[i])
        ax[i].set_ylabel(ylabels[i])
        ax[i].legend(loc='upper right')
        
    ax[3].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

### Multimodal DTF analysis for EEG and IBI signals

In [ ]:
f = np.arange(0.01, 1, 0.01)
# Channel names for visualization
ChanNames = ['Ch IBI', 'Cg IBI', 'Ch Fz\ntheta amp', 'Cg Fz_cg\ntheta amp']

for event in selected_events:
    # Estimate multivariate spectra and DTF for the 4-variable system
    S = multivariate_spectra(DTF_data, f, Fs=filtered_data['Fs_IBI'], max_p=15, p_opt=None, crit_type='AIC')
    DTF = DTF_multivariate(DTF_data, f, Fs=filtered_data['Fs_IBI'], max_p=15, p_opt=None, crit_type='AIC')
    
    # Plot DTF results in table form
    mvar_plot(S, DTF, f, 'From', 'To', ChanNames, f'DTF {event}', 'sqrt')
    plt.show()
    
    # Plot DTF results in graph form for selected frequency range
    fig, ax = plt.subplots(figsize=(10, 8))
    graph_plot(connectivity_matrix=DTF, ax=ax, f=f, f_range=[0.2, 0.6], 
                ChanNames=ChanNames, title=f'DTF {event}')
    plt.show()

## Summary and Conclusions

This notebook demonstrated a comprehensive analysis of hyperscanning EEG data including:

### Key Analysis Steps:
1. **Data Loading**: Real EEG/ECG data from child-caregiver dyads
2. **Preprocessing**: Filtering, artifact removal, and event detection
3. **Visualizations**
4. **Connectivity Analysis**: Directed Transfer Function (DTF) for multiple modalities

### Analysis Types:
- **HRV Connectivity**: Inter-subject heart rate variability coupling
- **EEG Connectivity**: Within-subject EEG network analysis  
- **Multimodal Connectivity**: EEG theta activity and HRV interactions

### Technical Features:
- IBI extraction and interpolation from ECG
- Theta band filtering and instantaneous amplitude


The analysis reveals connectivity patterns between physiological signals during naturalistic hyperscanning experiments, providing insights into dyadic coupling mechanisms.